In [ ]:
%load_ext autoreload
%autoreload 2

import sys
import os
import time

import numpy as np
import mxnet as mx
import matplotlib.pyplot as plt
%matplotlib inline

sys.path.append(os.environ['REPO_DIR'] + '/utilities')
from utilities2015 import *
from metadata import *
from data_manager import *
from learning_utilities import *

In [2]:
import logging
reload(logging) # This is important for logging to work on workstation jupyter notebook.
# See https://stackoverflow.com/questions/18786912/get-output-from-the-logging-module-in-ipython-notebook/21475297#21475297
head = '%(asctime)-15s %(message)s'
logging.basicConfig(level=logging.DEBUG, format=head)

In [3]:
classifier_id = 71
classifier_properties = classifier_settings.loc[classifier_id]

margin = classifier_properties['margin']
# model = classifier_properties['model']
sample_weighting = classifier_properties['sample_weighting']
neg_composition = classifier_properties['neg_composition']

In [4]:
model_dir_name = 'inception-bn-blue-softmax'
download_from_s3(os.path.join(MXNET_MODEL_ROOTDIR, model_dir_name), is_dir=True)
model_name = 'inception-bn-blue-softmax'
model_iteration = 0
output_symbol_name = 'softmax_output'
output_dim = 1024
mean_img = np.load(os.path.join(MXNET_MODEL_ROOTDIR, model_dir_name, 'mean_224.npy'))

# Reference on how to predict with mxnet model:
# https://github.com/dmlc/mxnet-notebooks/blob/master/python/how_to/predict.ipynb
model_prefix = os.path.join(MXNET_MODEL_ROOTDIR, model_dir_name, model_name)
model0, arg_params, aux_params = mx.model.load_checkpoint(model_prefix, 0)

In [5]:
# Finetune
# http://mxnet.io/how_to/finetune.html

In [6]:
def get_fine_tune_model(symbol, arg_params, num_classes, layer_name='flatten'):
    """
    symbol: the pretrained network symbol
    arg_params: the argument parameters of the pretrained model
    num_classes: the number of classes for the fine-tune datasets
    layer_name: the layer name before the last fully-connected layer
    """
    all_layers = symbol.get_internals()
    net = all_layers[layer_name + '_output']
    net = mx.symbol.FullyConnected(data=net, num_hidden=num_classes, name='fc1')
    net = mx.symbol.SoftmaxOutput(data=net, name='softmax')
    new_args = {k: arg_params[k] for k in arg_params if 'fc1' not in k}
    return (net, new_args)

In [7]:
num_classes = 2
(new_sym, new_args) = get_fine_tune_model(model0, arg_params, num_classes)

In [8]:
def fit(symbol, arg_params, aux_params, train, val, batch_size, num_gpus, num_epoch, epoch_end_callback,
       eval_end_callback):
    devs = [mx.gpu(i) for i in range(num_gpus)]
    mod = mx.mod.Module(symbol=symbol, context=devs)
    mod.fit(train, val,
        num_epoch=num_epoch,
        arg_params=arg_params,
        aux_params=aux_params,
        allow_missing=True,
        batch_end_callback = mx.callback.Speedometer(batch_size, 10),
        kvstore='device',
        optimizer='sgd',
        optimizer_params={'learning_rate':0.01},
        initializer=mx.init.Xavier(rnd_type='gaussian', factor_type="in", magnitude=2),
        eval_metric='acc',
           epoch_end_callback=epoch_end_callback,
            eval_end_callback=eval_end_callback
           )
    metric = mx.metric.Accuracy()
    return mod.score(val, metric)

In [9]:
batch_per_gpu = 16
num_gpus = 1
batch_size = batch_per_gpu * num_gpus

In [10]:
from mxnet.model import save_checkpoint

def my_epoch_end_callback(prefix, period=1):
    def _callback(epoch, sym, arg, aux):    
        if epoch % period == 0:
            save_checkpoint(prefix, epoch, sym, arg, aux)
            symbol_fp = '%s-symbol.json' % prefix
            param_fp = '%s-%04d.params' % (prefix, epoch)
            upload_to_s3(symbol_fp)
            upload_to_s3(param_fp)
    return _callback

def my_eval_end_callback(eval_metric_history):
    """
    Args:
        eval_metric_history (dict): {name: list of values}
    """
    def _callback(param):
        if not param.eval_metric:
            return
        name_value = param.eval_metric.get_name_value()
        for name, value in name_value:
            logging.info('Epoch[%d] Validation-%s=%f', param.epoch, name, value)
            if name not in eval_metric_history:
                eval_metric_history[name] = []
            eval_metric_history[name].append(value)
#             with open(acc_fp, 'a') as f:
#                 f.write('Epoch[%d] Validation-%s=%f\n' % (param.epoch, name, value))
    return _callback

In [ ]:
# for structure in structures_found:
for structure in all_known_structures:
# for structure in ['SC']:

    try:

        print structure

        # Determine which labels to load.

        structures_to_sample = [structure]
        negative_labels_to_sample = [s + '_negative' for s in structures_to_sample]

        margins_to_sample = [margin] # (200: 100 um, 500: 250 um)
        surround_positive_labels_to_sample = [convert_to_surround_name(s, margin=m, suffix=surr_l) 
                                     for m in margins_to_sample
                                     for s in structures_to_sample 
                                     for surr_l in all_known_structures
                                     if surr_l != s]
        surround_noclass_labels_to_sample = [convert_to_surround_name(s, margin=m, suffix='noclass') 
                                     for m in margins_to_sample
                                     for s in structures_to_sample]

        if neg_composition == 'neg_has_everything_else':
            labels_to_sample = structures_to_sample + negative_labels_to_sample
        elif neg_composition == 'neg_has_only_surround_noclass':
            labels_to_sample = structures_to_sample + surround_noclass_labels_to_sample + ['noclass']
        elif neg_composition == 'neg_has_all_surround':
            labels_to_sample = structures_to_sample + surround_positive_labels_to_sample + surround_noclass_labels_to_sample + ['noclass']

        # labels_to_sample = ['Sp5C', 'Sp5C_surround_500_Sp5I', 'Sp5C_surround_500_noclass', 'Sp5C_surround_500_LRt']
        # labels_to_sample = ['Sp5O', 'Sp5O_surround_500_Sp5I', 'Sp5O_surround_500_noclass']
#         labels_to_sample = ['SC', 'SC_surround_500_IC', 'SC_surround_500_noclass']

        # Load training dataset.

        training_set_ids = map(int, str(classifier_properties['train_set_id']).split('/'))
        training_features, training_addresses = load_dataset_images(training_set_ids, labels_to_sample=labels_to_sample)

        # convert patches data shape to nx1x224x224
        training_features = {s: (patches - mean_img)[:, None, :, :] for s, patches in training_features.iteritems()}

        # check which labels are collected
        labels_found = training_addresses.keys()
        structures_found = set([convert_to_original_name(l) for l in labels_found 
                                if convert_to_original_name(l) in labels_found]) - {'noclass'}

        # Load test dataset.

#         test_set_ids = [62]
#         test_features, test_addresses = load_dataset_images(test_set_ids, labels_to_sample=labels_to_sample)
#         test_features = {s: (patches - mean_img)[:, None, :, :] for s, patches in test_features.iteritems()}

        #############################################


        if neg_composition == 'neg_has_only_surround_noclass':
            neg_classes = [convert_to_surround_name(structure, margin=margin, suffix='noclass')]
        elif neg_composition == 'neg_has_all_surround':
            neg_classes = [convert_to_surround_name(structure, margin=margin, suffix='noclass')]
            for surr_s in structures_found:
                c = convert_to_surround_name(structure, margin=margin, suffix=surr_s)
                if c in labels_found:
                    neg_classes.append(c)
        elif neg_composition == 'neg_has_everything_else':
            neg_classes = [structure + '_negative']
        else:
            raise Exception('neg_composition %s is not recognized.' % neg_composition)

        ###########################
        ## Define Sample Weights ##
        ###########################

    #     if sample_weighting == 'weighted':
    #         neg_distances = np.concatenate([distances_to_structures[neg_class][structure] for neg_class in neg_classes])

    #         sample_weights_neg = np.ones((n_neg,))
    #         sample_weights_neg[neg_distances > thresh] = diminishing(neg_distances[neg_distances > thresh])
    #         sample_weights = np.r_[np.ones((n_pos,)), sample_weights_neg]
    #     else:
    #         sample_weights = None

        ###########################################################################################

        train_features_pos = training_features[structure]
        n_pos = len(train_features_pos)

        train_features_neg = np.concatenate([training_features[neg_class] for neg_class in neg_classes])
        n_neg = len(train_features_neg)

        train_data = np.concatenate([train_features_pos, train_features_neg])
        # For cnn, labels must be 0/1 rather than +1/-1
        train_labels = np.r_[np.ones((n_pos, )), np.zeros((n_neg, ))]

        train_data_iter = mx.io.NDArrayIter(
            data=train_data, 
            batch_size=batch_size,
            label=train_labels,
            shuffle=True)

        #####################################

#         test_features_pos = test_features[structure]
#         n_pos = len(test_features_pos)

#         test_features_neg = np.concatenate([test_features[neg_class] for neg_class in neg_classes \
#                                        if neg_class in test_features])
#         n_neg = len(test_features_neg)

#         test_data = np.concatenate([test_features_pos, test_features_neg])
#         # For cnn, labels must be 0/1 rather than +1/-1
#         test_labels = np.r_[np.ones((n_pos, )), np.zeros((n_neg, ))]

#         test_data_iter = mx.io.NDArrayIter(
#             data=test_data, 
#             batch_size=batch_size,
#             label=test_labels,
#             shuffle=True)

        eval_metric_history = {}

        t = time.time()
        prefix = os.path.join(MXNET_MODEL_ROOTDIR, model_dir_name, model_name + '_' + structure)
#         mod_score = fit(new_sym, new_args, aux_params, train_data_iter, test_data_iter, 
#                         batch_size, num_gpus, num_epoch=10, epoch_end_callback=my_epoch_end_callback(prefix, period=5))
        mod_score = fit(new_sym, new_args, aux_params, train_data_iter, train_data_iter, 
                        batch_size, num_gpus, num_epoch=50, 
                        epoch_end_callback=my_epoch_end_callback(prefix, period=5),
                       eval_end_callback=my_eval_end_callback(eval_metric_history))
        sys.stderr.write('Fitting classifier: %.2f seconds\n' % (time.time() - t))
        
        save_pickle( eval_metric_history, prefix+'_evalMetricHistory.pkl')
        upload_to_s3(prefix+'_evalMetricHistory.pkl')

    #     clf_fp = DataManager.get_classifier_filepath(classifier_id=classifier_id, structure=structure)
    #     create_parent_dir_if_not_exists(clf_fp)
    #     joblib.dump(clf, clf_fp)
    #     upload_to_s3(clf_fp)
    except Exception as e:
        sys.stderr.write("Skip %s: %s.\n" % (structure, str(e)))